In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import Point
import numpy as np

# Load the NHGIS data table
df_table = pd.read_csv(
    r"nhgis0004_ds267_20235_tract.csv",
    low_memory=False
)

# Set all negative values to 0 in mediam household income (this became an issue much later on)
df_table['ASQPE001'] = df_table['ASQPE001'].where(df_table['ASQPE001'] >= 0, 0)

# Remove the low_memory=False parameter and try with default settings
gis_shap = gpd.read_file(r"US_tract_2023.shp")

# Immediately filter to Michigan to reduce memory usage
gis_merged = gis_shap.merge(df_table, on='GISJOIN')
gis_mi = gis_merged[gis_merged['STATE'] == 'Michigan'].copy()

# Delete the large objects to free memory
del gis_merged, gis_shap, df_table

# Load SEMCOG dataset
semcog = pd.read_csv(r"crash2024_10year_-2940358597150127354.csv", low_memory=False)
semcog = semcog[semcog['YEAR'].isin([2023, 2022, 2021, 2020, 2019])]

g = [Point(xy) for xy in zip(semcog['XCORD'], semcog['YCORD'])]

# EPSG:4326 is the CRS I useed because it is the standard fot GCS system, which use the latitude/longitude sysetm present in the semcog data set
semcog_gdf = gpd.GeoDataFrame(semcog, geometry=g, crs="EPSG:4326")

# here I set the coordinate control system (CRS) of the semcog data set to the CRS  of the gis_mi data
semcog_gdf = semcog_gdf.to_crs(gis_mi.crs)

# Spatial join to assign tracts, used predicate within because I only wanted the ones, which were, well... within
semcog_new = gpd.sjoin(semcog_gdf, gis_mi, how="left", predicate="within")
del g, semcog_gdf, gis_mi, semcog

# Dropped all the columns related to Margin of Error to make data easier to intrepet and load
moe_cols_to_drop = [
    'NAME_M',
    'ASOAM001', 'ASOAM002', 'ASOAM003', 'ASOAM004', 'ASOAM005', 'ASOAM006', 'ASOAM007', 'ASOAM008', 'ASOAM009', 'ASOAM010',
    'ASOAM011', 'ASOAM012', 'ASOAM013', 'ASOAM014', 'ASOAM015', 'ASOAM016', 'ASOAM017', 'ASOAM018', 'ASOAM019', 'ASOAM020', 'ASOAM021',
    'ASQIM001', 'ASQIM002', 'ASQIM003', 'ASQIM004', 'ASQIM005', 'ASQIM006', 'ASQIM007', 'ASQIM008',
    'ASQOM001', 'ASQOM002', 'ASQOM003', 'ASQOM004', 'ASQOM005', 'ASQOM006', 'ASQOM007', 'ASQOM008', 'ASQOM009', 'ASQOM010',
    'ASQOM011', 'ASQOM012', 'ASQOM013', 'ASQOM014', 'ASQOM015', 'ASQOM016', 'ASQOM017',
    'ASQPM001'
]

semcog_new = semcog_new.drop(columns=moe_cols_to_drop, errors='ignore')
del moe_cols_to_drop

# I renamed all my columns for simplicity. The metadata included what each column stood for and thus I used definations from there
def rename_columns(df):
    rename_dict = {
        # Hispanic or Latino Origin by Race (Estimates)
        'ASOAE001': 'Total_Population',
        'ASOAE002': 'Not_Hispanic_or_Latino',
        'ASOAE003': 'White',
        'ASOAE004': 'Black',
        'ASOAE005': 'Not_Hispanic_AIAN_Alone',
        'ASOAE006': 'Asian',
        'ASOAE007': 'NHPI',
        'ASOAE008': 'Not_Hispanic_Other_Race_Alone',
        'ASOAE009': 'Not_Hispanic_Two_or_More_Races',
        'ASOAE010': 'Not_Hispanic_Two_Races_SomeOther',
        'ASOAE011': 'Not_Hispanic_Two_Races_Exclude_SomeOther',
        'ASOAE012': 'Hispanic_or_Latino',
        'ASOAE013': 'Hispanic_White',
        # Keep individual columns for now - we'll combine them after renaming
        'ASOAE014': 'Hispanic_Black_Alone',
        'ASOAE015': 'Hispanic_AIAN_Alone',
        'ASOAE016': 'Hispanic_Asian_Alone',
        'ASOAE017': 'Hispanic_NHPI_Alone',
        'ASOAE018': 'Hispanic_Other_Race_Alone',
        'ASOAE019': 'Hispanic_Two_or_More_Races',
        'ASOAE020': 'Hispanic_Two_Races_SomeOther',
        'ASOAE021': 'Hispanic_Two_Races_Exclude_SomeOther',
        # Ratio of Income to Poverty Level
        'ASQIE001': 'Poverty_Total',
        'ASQIE002': 'Under_50_Percent_Poverty',
        'ASQIE003': '50_to_99_Percent_Poverty',
        'ASQIE004': '100_to_124_Percent_Poverty',
        'ASQIE005': '125_to_149_Percent_Poverty',
        'ASQIE006': '150_to_184_Percent_Poverty',
        'ASQIE007': '185_to_199_Percent_Poverty',
        'ASQIE008': '200_Percent_or_More_Poverty',
        # Household (HH) Income Ranges
        'ASQOE001': 'HH_Income_Total',
        'ASQOE002': 'HH_Income_Less_Than_10K',
        'ASQOE003': 'HH_Income_10K_to_14_999',
        'ASQOE004': 'HH_Income_15K_to_19_999',
        'ASQOE005': 'HH_Income_20K_to_24_999',
        'ASQOE006': 'HH_Income_25K_to_29_999',
        'ASQOE007': 'HH_Income_30K_to_34_999',
        'ASQOE008': 'HH_Income_35K_to_39_999',
        'ASQOE009': 'HH_Income_40K_to_44_999',
        'ASQOE010': 'HH_Income_45K_to_49_999',
        'ASQOE011': 'HH_Income_50K_to_59_999',
        'ASQOE012': 'HH_Income_60K_to_74_999',
        'ASQOE013': 'HH_Income_75K_to_99_999',
        'ASQIE014': 'HH_Income_100K_to_124_999',
        'ASQOE015': 'HH_Income_125K_to_149_999',
        'ASQOE016': 'HH_Income_150K_to_199_999',
        'ASQOE017': 'HH_Income_200K_or_More',
        # Median Household Income
        'ASQPE001': 'Median_HH_Income_2023'
    }

    return df.rename(columns=rename_dict)

# Running rename fucntion above
semcog_new = rename_columns(semcog_new)

# Combine all hispanic non-white columns
hispanic_non_white_cols = [
    'Hispanic_Black_Alone',
    'Hispanic_AIAN_Alone',
    'Hispanic_Asian_Alone',
    'Hispanic_NHPI_Alone',
    'Hispanic_Other_Race_Alone',
    'Hispanic_Two_or_More_Races',
    'Hispanic_Two_Races_SomeOther',
    'Hispanic_Two_Races_Exclude_SomeOther'
]
existing_hispanic_cols = [c for c in hispanic_non_white_cols if c in semcog_new.columns]

if existing_hispanic_cols:
    semcog_new['Hispanic_non_White'] = (
        semcog_new[existing_hispanic_cols].fillna(0).sum(axis=1)
    )
else:
    semcog_new['Hispanic_non_White'] = 0
# Sum the Hispanic non-White columns (fill NaN with 0 first to handle missing values)
semcog_new['Hispanic_non_White'] = semcog_new[hispanic_non_white_cols].fillna(0).sum(axis=1)

# Create semcog_new_2 with percentage calculations
semcog_new_2 = semcog_new.copy()

del hispanic_non_white_cols

# Calculate percentages for all demographic variables (divide by Total_Population)
demographic_cols = [
    'Total_Population',
    'Not_Hispanic_or_Latino',
    'Not_Hispanic_White_Alone',
    'Not_Hispanic_Black_Alone',
    'Not_Hispanic_AIAN_Alone',
    'Not_Hispanic_Asian_Alone',
    'Not_Hispanic_NHPI_Alone',
    'Not_Hispanic_Other_Race_Alone',
    'Not_Hispanic_Two_or_More_Races',
    'Not_Hispanic_Two_Races_SomeOther',
    'Not_Hispanic_Two_Races_Exclude_SomeOther',
    'Hispanic_or_Latino',
    'Hispanic_White',
    'Hispanic_non_White'
]

# Calculate demographic percentages
for col in demographic_cols:
    if col in semcog_new_2.columns:
        semcog_new_2[f'{col}_Percent_values'] = (semcog_new_2[col] / semcog_new_2['Total_Population']) * 100
del demographic_cols

# Calculate percentages for poverty variables (divide by Poverty_Total)
poverty_cols = [
    'Under_50_Percent_Poverty',
    '50_to_99_Percent_Poverty',
    '100_to_124_Percent_Poverty',
    '125_to_149_Percent_Poverty',
    '150_to_184_Percent_Poverty',
    '185_to_199_Percent_Poverty',
    '200_Percent_or_More_Poverty'
]

# Calculate poverty percentages
for col in poverty_cols:
    if col in semcog_new_2.columns:
        semcog_new_2[f'{col}_Percent_values'] = (semcog_new_2[col] / semcog_new_2['Poverty_Total']) * 100
del poverty_cols

# Calculate percentages for household income variables (divide by HH_Income_Total)
hh_income_cols = [
    'HH_Income_Less_Than_10K',
    'HH_Income_10K_to_14_999',
    'HH_Income_15K_to_19_999',
    'HH_Income_20K_to_24_999',
    'HH_Income_25K_to_29_999',
    'HH_Income_30K_to_34_999',
    'HH_Income_35K_to_39_999',
    'HH_Income_40K_to_44_999',
    'HH_Income_45K_to_49_999',
    'HH_Income_50K_to_59_999',
    'HH_Income_60K_to_74_999',
    'HH_Income_75K_to_99_999',
    'HH_Income_100K_to_124_999',
    'HH_Income_125K_to_149_999',
    'HH_Income_150K_to_199_999',
    'HH_Income_200K_or_More'
]

# Calculate household income percentages
for col in hh_income_cols:
    if col in semcog_new_2.columns:
        semcog_new_2[f'{col}_Percent_values'] = (semcog_new_2[col] / semcog_new_2['HH_Income_Total']) * 100
del hh_income_cols
# Handle division by replacing NaN values with 0
semcog_new_2 = semcog_new_2.fillna(0)

# Because now thhey are over 200 columns I decided to drop the oroginal colums adn only keep percent ones
moe_cols_to_drop_2 = [
    'HH_Income_Less_Than_10K',
    'HH_Income_10K_to_14_999',
    'HH_Income_15K_to_19_999',
    'HH_Income_20K_to_24_999',
    'HH_Income_25K_to_29_999',
    'HH_Income_40K_to_44_999',
    'HH_Income_30K_to_34_999',
    'HH_Income_35K_to_39_999',
    'HH_Income_45K_to_49_999',
    'HH_Income_50K_to_59_999',
    'HH_Income_60K_to_74_999',
    'HH_Income_75K_to_99_999',
    'HH_Income_100K_to_124_999',
    'HH_Income_125K_to_149_999',
    'HH_Income_150K_to_199_999',
    'HH_Income_200K_or_More',
    'Under_50_Percent_Poverty',
    '50_to_99_Percent_Poverty',
    '100_to_124_Percent_Poverty',
    '125_to_149_Percent_Poverty',
    '150_to_184_Percent_Poverty',
    '185_to_199_Percent_Poverty',
    '200_Percent_or_More_Poverty',
    'Total_Population',
    'Not_Hispanic_or_Latino',
    'Not_Hispanic_White_Alone',
    'Not_Hispanic_Black_Alone',
    'Not_Hispanic_AIAN_Alone',
    'Not_Hispanic_Asian_Alone',
    'Not_Hispanic_NHPI_Alone',
    'Not_Hispanic_Other_Race_Alone',
    'Not_Hispanic_Two_or_More_Races',
    'Not_Hispanic_Two_Races_SomeOther',
    'Not_Hispanic_Two_Races_Exclude_SomeOther',
    'Hispanic_White',
    'Hispanic_non_White',
    'Hispanic_Black_Alone',
    'Hispanic_AIAN_Alone',
    'Hispanic_Asian_Alone',
    'Hispanic_NHPI_Alone',
    'Hispanic_Other_Race_Alone',
    'Hispanic_Two_or_More_Races',
    'Hispanic_Two_Races_SomeOther',
    'Hispanic_Two_Races_Exclude_SomeOther',
    'STATEFP',
    'GEOID',
    'GEOIDFQ',
    'MTFCC',
    'FUNCSTAT',
    'DATE_FULL',
    'YEAR_left',
    'MONTH',
    'x',
    'y',
    'ALAND',
    'AWATER',
    'INTPTLAT',
    'INTPTLON',
    'Shape_Leng',
    'Shape_Area',
    'ORIG_FID',
    'YEAR_right',
    'STUSAB',
    'REGIONA',
    'DIVISIONA',
    'STATE',
    'STATEA',
    'COUSUBA',
    'PLACEA',
    'BLKGRPA',
    'CONCITA',
    'AIANHHA',
    'RES_ONLYA',
    'TRUSTA',
    'AIHHTLI',
    'AITSA',
    'ANRCA',
    'CBSAA',
    'CSAA',
    'METDIVA',
    'CNECTA',
    'NECTADIV',
    'UAA',
    'CDCURRA',
    'SLDUA',
    'SLDLA',
    'ZCTA5A',
    'SUBMCDA',
    'SDELMA',
    'SDSECA',
    'SDUNIA',
    'PCI',
    'PUMAA',
    'GEO_ID',
    'BTTRA',
    'BTBGA'
]
semcog_new_2 = semcog_new_2.drop(columns=moe_cols_to_drop_2, errors='ignore')
del moe_cols_to_drop_2

# Next part I find average crash rates, and most common race, income level, etc. for each row
def analyze_intersection(semcog_new_2, road1, road2):

    # Get the target intersection data
    target_intersection = semcog_new_2[
        ((semcog_new_2['INTERROAD'].str.contains(road1, case=False, na=False)) &
         (semcog_new_2['MAINROAD'].str.contains(road2, case=False, na=False))) |
        ((semcog_new_2['INTERROAD'].str.contains(road2, case=False, na=False)) &
         (semcog_new_2['MAINROAD'].str.contains(road1, case=False, na=False)))
    ].copy()

    def determine_income_level(row):
        """Calculate mean income level based on household income percentages"""
        income_cols = [
            'HH_Income_Less_Than_10K_Percent_values',
            'HH_Income_10K_to_14_999_Percent_values',
            'HH_Income_15K_to_19_999_Percent_values',
            'HH_Income_20K_to_24_999_Percent_values',
            'HH_Income_25K_to_29_999_Percent_values',
            'HH_Income_30K_to_34_999_Percent_values',
            'HH_Income_35K_to_39_999_Percent_values',
            'HH_Income_40K_to_44_999_Percent_values',
            'HH_Income_45K_to_49_999_Percent_values',
            'HH_Income_50K_to_59_999_Percent_values',
            'HH_Income_60K_to_74_999_Percent_values',
            'HH_Income_75K_to_99_999_Percent_values',
            'HH_Income_125K_to_149_999_Percent_values',
            'HH_Income_150K_to_199_999_Percent_values',
            'HH_Income_200K_or_More_Percent_values'
        ]

        income_values = []
        for col in income_cols:
            if col in row.index and pd.notna(row[col]):
                income_values.append(row[col])


    def get_top_races(row):
        """Get top 2 most common races"""
        race_cols = ['White', 'Black', 'Asian', 'NHPI', 'Hispanic_or_Latino']
        race_values = {}

        for col in race_cols:
            if col in row.index:
                race_values[col] = row[col]

        # Sort by values (descending)
        sorted_races = sorted(race_values.items(), key=lambda x: x[1], reverse=True)

        # Get top 2
        top_2 = [race[0] for race in sorted_races[:2]]
        return top_2


    def get_top_income_brackets(row):
        """Get top 3 income brackets"""
        income_cols = [
            'HH_Income_Less_Than_10K_Percent_values',
            'HH_Income_10K_to_14_999_Percent_values',
            'HH_Income_15K_to_19_999_Percent_values',
            'HH_Income_20K_to_24_999_Percent_values',
            'HH_Income_25K_to_29_999_Percent_values',
            'HH_Income_30K_to_34_999_Percent_values',
            'HH_Income_35K_to_39_999_Percent_values',
            'HH_Income_40K_to_44_999_Percent_values',
            'HH_Income_45K_to_49_999_Percent_values',
            'HH_Income_50K_to_59_999_Percent_values',
            'HH_Income_60K_to_74_999_Percent_values',
            'HH_Income_75K_to_99_999_Percent_values',
            'HH_Income_125K_to_149_999_Percent_values',
            'HH_Income_150K_to_199_999_Percent_values',
            'HH_Income_200K_or_More_Percent_values'
        ]

        income_values = {}
        for col in income_cols:
            if col in row.index:
                income_values[col] = row[col]

        # Sort by values (descending)
        sorted_income = sorted(income_values.items(), key=lambda x: x[1], reverse=True)

        # Get top 3 and map to readable names
        income_mapping = {
            'HH_Income_Less_Than_10K_Percent_values': 'Under $10K',
            'HH_Income_10K_to_14_999_Percent_values': '$10K-$14,999',
            'HH_Income_15K_to_19_999_Percent_values': '$15K-$19,999',
            'HH_Income_20K_to_24_999_Percent_values': '$20K-$24,999',
            'HH_Income_25K_to_29_999_Percent_values': '$25K-$29,999',
            'HH_Income_30K_to_34_999_Percent_values': '$30K-$34,999',
            'HH_Income_35K_to_39_999_Percent_values': '$35K-$39,999',
            'HH_Income_40K_to_44_999_Percent_values': '$40K-$44,999',
            'HH_Income_45K_to_49_999_Percent_values': '$45K-$49,999',
            'HH_Income_50K_to_59_999_Percent_values': '$50K-$59,999',
            'HH_Income_60K_to_74_999_Percent_values': '$60K-$74,999',
            'HH_Income_75K_to_99_999_Percent_values': '$75K-$99,999',
            'HH_Income_125K_to_149_999_Percent_values': '$125K-$149,999',
            'HH_Income_150K_to_199_999_Percent_values': '$150K-$199,999',
            'HH_Income_200K_or_More_Percent_values': '$200K+'
        }

        top_3 = [income_mapping.get(item[0], item[0]) for item in sorted_income[:3]]
        return top_3

    def get_poverty_level(row):
        """Calculate weighted average poverty level and return the corresponding income range"""
        poverty_cols_with_ranges = [
            ('Under_50_Percent_Poverty_Percent_values', 'Under 50% of Poverty Level'),
            ('50_to_99_Percent_Poverty_Percent_values', '50-99% of Poverty Level'),
            ('100_to_124_Percent_Poverty_Percent_values', '100-124% of Poverty Level'),
            ('125_to_149_Percent_Poverty_Percent_values', '125-149% of Poverty Level'),
            ('150_to_184_Percent_Poverty_Percent_values', '150-184% of Poverty Level'),
            ('185_to_199_Percent_Poverty_Percent_values', '185-199% of Poverty Level'),
            ('200_Percent_or_More_Poverty_Percent_values', '200%+ of Poverty Level')
        ]

        # Assign midpoint values to each poverty bracket for weighted average calculation
        poverty_midpoints = [25, 74.5, 112, 137, 167, 192, 250]  # Midpoints of each bracket

        total_weighted_value = 0
        total_percentage = 0

        for i, (col, range_name) in enumerate(poverty_cols_with_ranges):
            if col in row.index and pd.notna(row[col]) and row[col] > 0:
                percentage = row[col]
                weighted_value = percentage * poverty_midpoints[i]
                total_weighted_value += weighted_value
                total_percentage += percentage

        if total_percentage == 0:
            return "No poverty data available"

        # Calculate weighted average poverty level percentage
        weighted_avg_poverty_level = total_weighted_value / total_percentage

        # Map the weighted average back to the appropriate range
        if weighted_avg_poverty_level < 50:
            return "Under 50% of Poverty Level"
        elif weighted_avg_poverty_level < 100:
            return "50-99% of Poverty Level"
        elif weighted_avg_poverty_level < 125:
            return "100-124% of Poverty Level"
        elif weighted_avg_poverty_level < 150:
            return "125-149% of Poverty Level"
        elif weighted_avg_poverty_level < 185:
            return "150-184% of Poverty Level"
        elif weighted_avg_poverty_level < 200:
            return "185-199% of Poverty Level"
        else:
            return "200%+ of Poverty Level"

    def analyze_crashes_all_intersection(df):
        """Analyze crash rates using ALL crashes at the intersection"""
        if len(df) == 0:
            return {}

        # Crash-related columns for rate calculation
        crash_cols = [
            'PERCENT_INJURED', 'MOTORIST_INJURY', 'TOTAL_INJURY', 'BIKEINJURY',
            'PARTYCOUNT', 'SPEEDING', 'DRIVEWAY', 'WORK_ZONE', 'DISTRACTED',
            'EMERGENCY', 'COMMERCIAL', 'TRAIN', 'MOTORCYCLE', 'BICYCLE',
            'PEDESTRIAN', 'DRUG', 'ALCOHOL', 'HITNRUN', 'SCHOOLBUS', 'DEER'
        ]

        # Additional columns for averages (not rates)
        avg_cols = ['SPEEDLIMIT', 'ROADLANES']

        # Maximum AADTR (busier road determines intersection risk)
        aadtr_values = df['AADTR'][df['AADTR'] > 0]  # Exclude zero values
        if len(aadtr_values) == 0:
            return {}

        max_aadtr = aadtr_values.max()

        # Calculate crash statistics using ALL crashes at the intersection
        stats = {}

        # Calculate crash rates per AADTR for each crash type
        for col in crash_cols:
            if col in df.columns:
                if col in ['PERCENT_INJURED', 'PARTYCOUNT']:
                    # For these, calculate average value across all crashes
                    stats[f'avg_{col}'] = df[col].mean()
                else:
                    # For binary columns, calculate rate = (sum of crashes with this factor) / max_AADTR
                    crash_count = df[col].sum()
                    stats[f'rate_{col}'] = (crash_count / max_aadtr) * 1000  # Rate per 1000 AADTR

        # Calculate total crash rate for this intersection
        total_crashes = len(df)
        stats['rate_TOTAL_CRASHES'] = (total_crashes / max_aadtr) * 1000  # Rate per 1000 AADTR
        stats['total_crashes_at_intersection'] = total_crashes  # Raw count of all crashes at intersection

        # Calculate crash rate for crashes with NONE of the specific factors
        factor_cols = [col for col in crash_cols if col not in ['PERCENT_INJURED', 'PARTYCOUNT']]

        # Find crashes that have none of these factors (all zeros)
        crashes_with_no_factors = df[factor_cols].sum(axis=1) == 0
        crashes_no_factors_count = crashes_with_no_factors.sum()
        stats['rate_NO_FACTORS'] = (crashes_no_factors_count / max_aadtr) * 1000  # Rate per 1000 AADTR

        # Add averages for non-rate columns
        for col in avg_cols:
            if col in df.columns:
                stats[f'avg_{col}'] = df[col].mean()

        # Add the AADTR used for calculations
        stats['AADTR_used'] = max_aadtr

        return stats

    results = []

    if len(target_intersection) > 0:
        # For the target intersection, use ALL crashes regardless of AADTR
        # But still get most common track and NFC for finding similar intersections
        most_common_track = target_intersection['COUNTY'].mode().iloc[0] if len(target_intersection['COUNTY'].mode()) > 0 else target_intersection['COUNTY'].iloc[0]
        most_common_nfc = target_intersection['NFC'].mode().iloc[0] if len(target_intersection['NFC'].mode()) > 0 else target_intersection['NFC'].iloc[0]

        # Skip if all AADTR values are 0
        valid_aadtr = target_intersection['AADTR'][target_intersection['AADTR'] > 0]
        if len(valid_aadtr) == 0:
            return results

        # Target intersection analysis using ALL crashes
        target_avg = target_intersection.mean(numeric_only=True)
        target_analysis = {
            'Location': f'{road1}-{road2}',
            'AADTR': valid_aadtr.max(),  # Use the max AADTR from the intersection
            'NFC': most_common_nfc,
            'Top_2_Races': get_top_races(target_avg),
            'Top_3_Income_Brackets': get_top_income_brackets(target_avg),
            'Mean_Poverty_Level': get_poverty_level(target_avg),
            'Median_HH_Income': target_avg.get('Median_HH_Income_2023', 0),
            'Total_Crashes': len(target_intersection)
        }

        # Add crash statistics using ALL crashes at intersection
        crash_stats = analyze_crashes_all_intersection(target_intersection)
        target_analysis.update(crash_stats)
        results.append(target_analysis)

        # Find similar intersections by NFC, AADTR, and tract
        # Use the most common AADTR from the target intersection for matching
        most_common_aadtr = target_intersection['AADTR'].mode().iloc[0] if len(target_intersection['AADTR'].mode()) > 0 else target_intersection['AADTR'].iloc[0]

        similar_intersections = semcog_new_2[
            (semcog_new_2['COUNTY'] == most_common_track) &
            (semcog_new_2['NFC'] == most_common_nfc) &
            (semcog_new_2['AADTR'] == most_common_aadtr)
        ].copy()

        # Remove the target intersection from similar intersections
        target_mask = (
            ((similar_intersections['INTERROAD'].str.contains(road1, case=False, na=False)) &
             (similar_intersections['MAINROAD'].str.contains(road2, case=False, na=False))) |
            ((similar_intersections['INTERROAD'].str.contains(road2, case=False, na=False)) &
             (similar_intersections['MAINROAD'].str.contains(road1, case=False, na=False)))
        )

        rest_of_data = similar_intersections[~target_mask]

        # Similar intersections analysis
        if len(rest_of_data) > 0:
            rest_avg = rest_of_data.mean(numeric_only=True)
            rest_analysis = {
                'Location': f'Similar to {road1}-{road2}',
                'AADTR': most_common_aadtr,
                'NFC': most_common_nfc,
                'Top_2_Races': get_top_races(rest_avg),
                'Top_3_Income_Brackets': get_top_income_brackets(rest_avg),
                'Mean_Poverty_Level': get_poverty_level(rest_avg),
                'Median_HH_Income': rest_avg.get('Median_HH_Income_2023', 0),
                'Total_Crashes': len(rest_of_data)
            }

            # Add crash statistics for similar intersections
            crash_stats = analyze_crashes_all_intersection(rest_of_data)
            rest_analysis.update(crash_stats)
            results.append(rest_analysis)

    return results
all_results = pd.DataFrame()

# Analyze different intersections
intersections_to_analyze = [
    ('Hartland', 'Rovey'),
    ('Tienken', 'Sheldon'),
    ('Utica', 'Dodge'),
    ('Lee', 'SB'),
    ('Lee', 'Whitmore'),
    ('25', 'Hayes'),
    ('Maple', 'Drake'),
    ('Maple', 'Farmington'),
    ('BogieLake', 'CooleyLake'),
    ('Main', 'S3'),
    ('Loop', 'Commerce'),
    ('Cooley', 'Oxbow'),
    ('14', 'Farmington'),
    ('Martin', 'Library'),
    ('Martin', 'PGA'),
    ('Martin', 'Oakley'),
    ('14', 'Orchard'),
    ('Grand', 'Hudson'),
    ('Grand', 'Lyon'),
    ('Hudson', 'Pontiac'),
    ('White', 'Duck'),
    ('Hamlin', 'Livernois'),
    ('Tienken', 'Livernois'),
    ('Kercheval', 'Wayburn'),
    ('Ryder', 'Everett'),
    ('Ryder', 'Connors'),
    ('Firewood', 'Raintree'),
    ('Coachwood', 'Falcon'),
    ('Pioneer', 'Library'),
    ('Anthony', 'NB'),
    ('Anthony', 'SB'),
    ('Chilson', 'CoonLake'),
    ('Napier', '10'),
    ('Taft', 'Morgan'),
    ('Mile', 'Romeo'),
    ('19', 'Romeo'),
    ('Romeo', 'Cass'),
    ('Romeo', 'Canal'),
    ('Baldwin', 'Gregory'),
    ('Baldwin', 'Judah'),
    ('Hamburg', 'Winans'),
    ('Crescent', 'Town'),
    ('Village', 'Green'),
    ('Durant', 'Chayne'),
    ('Cole', 'Chevrolet'),
    ('Huron', 'Nixon'),
    ('Scio', 'Wagner'),
    ('Baker', 'DanHoey'),
    ('Bemis', 'Moon'),
    ('Whittaker', 'Merrit'),
    ('Textile', 'Hitchingham'),
    ('Textile', 'StonyCreek'),
    ('Whittaker', 'StonyCreek'),
    ('Geddes', 'Ridge'),
    ('Geddes', 'Superior'),
    ('Campus', 'Community'),
    ('Maple', 'WB'),
    ('Maple', 'EB'),
    ('Geddes', 'NB'),
    ('Geddes', 'SB'),
    ('Geddes', 'Earhart'),
    ('8', 'SB'),
    ('Territorial', 'NB'),
    ('Territorial', 'SB'),
    ('8Mile', 'Whitmore'),
    ('Franklin', '11Mile'),
    ('Yellowstone', 'Johnson'),
    ('Carriage', 'Glacier'),
    ('Pioneer', 'Meadow'),
    ('Kensington', 'Jacoby'),
    ('Bell', 'Coventry'),
    ('Metro', 'Sail'),
    ('33', 'McKay')
]

# Run analysis for each intersection and append results
for road1, road2 in intersections_to_analyze:
    intersection_results = analyze_intersection(semcog_new_2, road1, road2)
    if intersection_results:  # Only append if results found
        intersection_df = pd.DataFrame(intersection_results)
        all_results = pd.concat([all_results, intersection_df], ignore_index=True)

all_locations = set(all_results['Location'])

# Find which intersections have a "Similar to" counterpart
intersections_with_similar = set()

for location in all_locations:
    if not location.startswith('Similar to '):
        # Check if there's a corresponding "Similar to" entry
        similar_location = f'Similar to {location}'
        if similar_location in all_locations:
            intersections_with_similar.add(location)
            intersections_with_similar.add(similar_location)

# Filter the results to only include intersections that have both original and similar
filtered_results = all_results[all_results['Location'].isin(intersections_with_similar)].copy()

# Sort so that each intersection and its "Similar to" are grouped together
filtered_results = filtered_results.sort_values('Location')
all_results = filtered_results

del road1, road2, intersections_to_analyze, intersection_results, intersection_df, col, semcog_new, semcog_new_2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

def plot_total_injury_by_individual_income(df, title, color='steelblue'):
    """Plot total injury rates with individual columns for each household income level"""
    df_copy = df.copy()

    # Remove any rows with missing income data
    df_copy = df_copy.dropna(subset=['Median_HH_Income', 'rate_TOTAL_INJURY'])

    # Group by exact median household income and get the rate_TOTAL_INJURY
    # Use first() to get the rate for each income level (assuming one intersection per income level)
    grouped = df_copy.groupby('Median_HH_Income')['rate_TOTAL_INJURY'].first().sort_index()

    # Create the plot
    fig, ax = plt.subplots(1, 1, figsize=(20, 8))

    # Create bar chart with individual income levels
    x_positions = range(len(grouped))
    bars = ax.bar(x_positions, grouped.values, color=color,
                  edgecolor='white', linewidth=1.5, alpha=0.8)

    # Set title and labels
    ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Median Household Income', fontsize=14, fontweight='bold')
    ax.set_ylabel('Total Injury Rate per 1000 AADTR', fontsize=14, fontweight='bold')

    # Format x-axis labels to show income values
    income_labels = [f'${int(income/1000)}K' for income in grouped.index]
    ax.set_xticks(x_positions)
    ax.set_xticklabels(income_labels, rotation=45, ha='right', fontsize=10)

    # Add value labels on top of bars
    for i, (bar, value) in enumerate(zip(bars, grouped.values)):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(grouped.values)*0.01,
               f'{value:.2f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

    # Add grid for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.4)
    ax.set_axisbelow(True)

    # Improve layout
    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print(f"Number of income levels: {len(grouped)}")
    print(f"Income range: ${int(grouped.index.min()/1000)}K - ${int(grouped.index.max()/1000)}K")
    print(f"Total injury rate range: {grouped.min():.3f} - {grouped.max():.3f}")
    print(f"Mean total injury rate: {grouped.mean():.3f}")
    print(f"Median total injury rate: {grouped.median():.3f}")

    return grouped

def plot_total_injury_scatter_safe(df, title, color='steelblue'):
    """Plot total injury rates as scatter plot vs income with safe trend line calculation"""
    df_copy = df.copy().dropna(subset=['Median_HH_Income', 'rate_TOTAL_INJURY'])
    df_copy = df_copy.sort_values('Median_HH_Income')

    fig, ax = plt.subplots(1, 1, figsize=(12, 8))

    # Create scatter plot
    scatter = ax.scatter(df_copy['Median_HH_Income'], df_copy['rate_TOTAL_INJURY'],
                        c=color, s=100, alpha=0.7, edgecolors='white', linewidth=1.5)

    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Median Household Income ($)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Total Injury Rate per 1000 AADTR', fontsize=12, fontweight='bold')

    # Format x-axis to show income in thousands
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

    # Try to add trend line with error handling
    try:
        # Check if we have enough unique points and no constant values
        if len(df_copy) > 2 and df_copy['rate_TOTAL_INJURY'].std() > 1e-10:
            z = np.polyfit(df_copy['Median_HH_Income'], df_copy['rate_TOTAL_INJURY'], 1)
            p = np.poly1d(z)
            ax.plot(df_copy['Median_HH_Income'], p(df_copy['Median_HH_Income']),
                   "r--", alpha=0.8, linewidth=2, label='Trend Line')

            # Calculate correlation
            correlation = np.corrcoef(df_copy['Median_HH_Income'], df_copy['rate_TOTAL_INJURY'])[0,1]
            ax.text(0.05, 0.95, f'Correlation: {correlation:.3f}', transform=ax.transAxes,
                   fontsize=11, bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
            ax.legend(fontsize=10)
        else:
            print("Warning: Cannot compute trend line - insufficient variation in data")
    except (np.linalg.LinAlgError, ValueError) as e:
        print(f"Warning: Could not compute trend line due to numerical issues: {e}")

    ax.grid(True, linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()

# Separate unique and similar intersections from all_results
unique_intersections = all_results[~all_results['Location'].str.contains('Similar to', na=False)].copy()
similar_intersections = all_results[all_results['Location'].str.contains('Similar to', na=False)].copy()

print("=== TOTAL INJURY RATES BY INDIVIDUAL INCOME LEVELS ===")

# For unique intersections (roundabouts)
print("\n=== ROUNDABOUTS TOTAL INJURY ANALYSIS ===")
print(f"Number of unique intersections: {len(unique_intersections)}")

if len(unique_intersections) > 0:
    print("\n1. Total Injury Rates by Individual Income Levels:")
    unique_injury_data = plot_total_injury_by_individual_income(
        unique_intersections,
        'Total Injury Rates by Individual Median Household Income Levels (Roundabouts)',
        'steelblue'
    )

    print("\n2. Total Injury Rates vs Median Household Income (Scatter):")
    plot_total_injury_scatter_safe(
        unique_intersections,
        'Total Injury Rates vs Median Household Income (Roundabouts)',
        'steelblue'
    )
else:
    print("No unique intersections found in data")

# For similar intersections
print("\n=== SIMILAR INTERSECTIONS TOTAL INJURY ANALYSIS ===")
print(f"Number of similar intersections: {len(similar_intersections)}")

if len(similar_intersections) > 0:
    print("\n1. Total Injury Rates by Individual Income Levels:")
    similar_injury_data = plot_total_injury_by_individual_income(
        similar_intersections,
        'Total Injury Rates by Individual Median Household Income Levels (Similar Intersections)',
        'darkorange'
    )

    print("\n2. Total Injury Rates vs Median Household Income (Scatter):")
    plot_total_injury_scatter_safe(
        similar_intersections,
        'Total Injury Rates vs Median Household Income (Similar Intersections)',
        'darkorange'
    )
else:
    print("No similar intersections found in data")

# All intersections combined analysis
print("\n=== ALL INTERSECTIONS TOTAL INJURY ANALYSIS ===")
print(f"Number of all intersections: {len(all_results)}")

if len(all_results) > 0:
    print("\n1. Total Injury Rates by Individual Income Levels:")
    all_injury_data = plot_total_injury_by_individual_income(
        all_results,
        'Total Injury Rates by Individual Median Household Income Levels (All Intersections)',
        'darkgreen'
    )

    print("\n2. Total Injury Rates vs Median Household Income (Scatter):")
    plot_total_injury_scatter_safe(
        all_results,
        'Total Injury Rates vs Median Household Income (All Intersections)',
        'darkgreen'
    )
else:
    print("No intersection data found in all_results")
